In [24]:
from tqdm import tqdm
import pandas as pd
from sklearn.linear_model import LogisticRegression
from doc_vec_tools import label_sentences,train_dbow,get_vectors
from sklearn.model_selection import train_test_split
from gensim.models import Doc2Vec
import numpy as np
import random

In [42]:
# Ensure reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)

In [44]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [45]:
df = pd.read_json('dataset/train_set.json')
df['Text'] = df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
X_train, X_valid, y_train, y_valid = train_test_split(df.Text, df.label, random_state=0, test_size=0.2)
df_test = pd.read_json('./dataset/test_set.json')
df_test['Text'] = df_test.text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
X_test = df_test.Text
X_test = label_sentences(X_test, 'Test')
X_train = label_sentences(X_train, 'Train')
X_valid = label_sentences(X_valid, 'Valid')
all_data = X_train + X_test+X_valid

In [46]:
# Build doc2vec model
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8000/8000 [00:00<00:00, 2378734.72it/s]


In [47]:
# train doc2vec model
model_dbow = train_dbow(model = model_dbow, data = all_data,epochs=50)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8000/8000 [00:00<00:00, 3481111.32it/s]


In [48]:
from sklearn.metrics import accuracy_score
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
val_vectors_dbow = get_vectors(model_dbow, len(X_valid), 300, 'Valid')
test_vectors_dbow = get_vectors(model_dbow,len(X_test),300, 'Test')
logreg = LogisticRegression(n_jobs=1, C=1e5,max_iter=500)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(val_vectors_dbow)
print('Validation accuracy %s' % accuracy_score(y_pred, y_valid))

Validation accuracy 0.63125


In [33]:
predictions = logreg.predict(test_vectors_dbow)

In [34]:
# Write predictions to a file
import csv
with open("submission.csv-doc2vec", "w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','label'])
    for i, row in enumerate(predictions):
        csv_out.writerow([i, row])        